### This script converts position data from the DVL to lines of XML keypoints defining the trajectory

In [ ]:
import pandas as pd
dvl_df = pd.read_csv("dvl_position.csv")
dvl_df.head()

,t,__msgtype__,format,header.__msgtype__,header.frame_id,header.stamp.__msgtype__,header.stamp.nanosec,header.stamp.sec,pitch,roll,status,std,t_last_reset,time,x,y,yaw,z
0,1.724164e+09,sensors/msg/DVLPosition,json_v3.1,std_msgs/msg/Header,NaN,builtin_interfaces/msg/Time,801519155,1724164479,1.590327,-2.002819,0,0.0,1.724164e+09,1.687100e+09,0.051770,-0.014073,-0.632555,0.010189
1,1.724164e+09,sensors/msg/DVLPosition,json_v3.1,std_msgs/msg/Header,NaN,builtin_interfaces/msg/Time,6940126,1724164480,1.843149,-1.583715,0,0.0,1.724164e+09,1.687100e+09,0.064422,-0.009308,-0.610183,0.025066
2,1.724164e+09,sensors/msg/DVLPosition,json_v3.1,std_msgs/msg/Header,NaN,builtin_interfaces/msg/Time,221364498,1724164480,2.150046,-1.233694,0,0.0,1.724164e+09,1.687100e+09,0.073586,0.002362,-0.764366,0.047654
3,1.724164e+09,sensors/msg/DVLPosition,json_v3.1,std_msgs/msg/Header,NaN,builtin_interfaces/msg/Time,452746391,1724164480,2.295287,-0.836874,0,0.0,1.724164e+09,1.687100e+09,0.085261,0.014252,-1.169754,0.079026
4,1.724164e+09,sensors/msg/DVLPosition,json_v3.1,std_msgs/msg/Header,NaN,builtin_interfaces/msg/Time,660883903,1724164480,2.349584,-0.550021,0,0.0,1.724164e+09,1.687100e+09,0.100592,0.023341,-1.343815,0.113974


# From dvl_position.csv to XML keypoints

In [7]:
import numpy as np
import pandas as pd

# Params
dvl_file = "dvl_position.csv"
output_file = "keypoints.txt"
average_every = 2

convert_to_radians = True
time_start_at_zero = True

#  Axis map
M_xyz = np.array([
    [0, 0, 1],
    [-1, 0, 0],
    [0, -1, 0],
], dtype=float)

# Transform post
t_post = np.array([24.0, 0.0, 4.0], dtype=float)
rpy_offset = np.array([0.0, 0.0, 0.0], dtype=float)

# Read data
dvl_df = pd.read_csv(dvl_file)

usable_len = (len(dvl_df) // average_every) * average_every
dvl_df = dvl_df.iloc[:usable_len].copy()

groups = dvl_df.groupby(dvl_df.index // average_every, sort=True).mean(numeric_only=True)

if time_start_at_zero:
    groups["t"] -= groups["t"].iloc[0]

if convert_to_radians:
    groups["roll"]  = np.deg2rad(groups["roll"])
    groups["pitch"] = np.deg2rad(groups["pitch"])
    groups["yaw"]   = np.deg2rad(groups["yaw"])

xyz_out, rpy_out, time_out = [], [], []

for _, row in groups.iterrows():
    r_xyz = np.array([float(row["x"]), float(row["y"]), float(row["z"])], dtype=float)
    r_rpy = np.array([float(row["roll"]), float(row["pitch"]), float(row["yaw"])], dtype=float)

    xyz_mapped = M_xyz @ r_xyz + t_post
    rpy_mapped = M_xyz @ r_rpy + rpy_offset

    xyz_out.append(xyz_mapped)
    rpy_out.append(rpy_mapped)
    time_out.append(float(row["t"]))

xyz_out = np.asarray(xyz_out)
rpy_out = np.asarray(rpy_out)
time_out = np.asarray(time_out)

# To keypoints
fmt = lambda v: f"{v:.4f}"
lines = []
for i in range(len(time_out)):
    t = fmt(time_out[i])
    xyz = " ".join(fmt(v) for v in xyz_out[i])
    rpy = " ".join(fmt(v) for v in rpy_out[i])
    lines.append(f'<keypoint time="{t}" xyz="{xyz}" rpy="{rpy}"/>')

with open(output_file, "w") as f:
    f.write("\n".join(lines))


# DVL and depth to XML 
This uses pressure data to generate z positions

In [25]:
import numpy as np
import pandas as pd

# ---------------- Params ----------------
dvl_file = "dvl_position.csv"
depth_file = "depth.csv"
output_file = "keypoints.txt"
average_every = 2

convert_to_radians = True
time_start_at_zero = True

depth_column = "depth"       # kolonnen i depth.csv
depth_positive_down = True   # depth>0 nedover
world_Z_positive_up = True   # +Z oppover (ROS / ENU)
depth_offset_m = 0.0         # evt. sensor-offset

# Transformasjon og offset
t_post = (24.0, 0.0, 4.0)
rpy_offset = (0.0, 0.0, 0.0)

# ---------------- Les data ----------------
dvl_df = pd.read_csv(dvl_file)
depth_df = pd.read_csv(depth_file)

# Gjennomsnitt over "average_every"
usable_len = (len(dvl_df) // average_every) * average_every
dvl_df = dvl_df.iloc[:usable_len].copy()
groups = dvl_df.groupby(dvl_df.index // average_every, sort=True).mean(numeric_only=True)

# Start tid på null
if time_start_at_zero:
    t0 = groups["t"].iloc[0]
    groups["t"] -= t0
    depth_df = depth_df.copy()
    depth_df["t"] -= t0

# Konverter RPY til radianer
if convert_to_radians:
    groups["roll"]  = np.deg2rad(groups["roll"])
    groups["pitch"] = np.deg2rad(groups["pitch"])
    groups["yaw"]   = np.deg2rad(groups["yaw"])

# ---------------- Interpolér depth → DVL-tid ----------------
t_key = groups["t"].to_numpy()
t_depth = depth_df["t"].to_numpy()
depth_vals = depth_df[depth_column].to_numpy(dtype=float)

# Lineær interpolasjon (gir samme lengde som t_key)
depth_interp = np.interp(t_key, t_depth, depth_vals)

# Fortegn/offset
z_from_depth = depth_interp + depth_offset_m
if depth_positive_down and world_Z_positive_up:
    z_from_depth = -z_from_depth

# ---------------- Bygg keypoints ----------------
xyz_out, rpy_out, time_out = [], [], []

for i, row in groups.iterrows():
    # Hent DVL-data
    x_dvl = float(row["x"])
    y_dvl = float(row["y"])
    z_dvl = float(row["z"])
    roll  = float(row["roll"])
    pitch = float(row["pitch"])
    yaw   = float(row["yaw"])
    t     = float(row["t"])

    # Z fra depth i verdensramma (allerede med riktig fortegn)
    z_world_from_depth = float(z_from_depth[i])

    # Mapping tilsvarer (z, -x, -y), men vi OVERSTYRER kun Z_world
    Xw = z_dvl + t_post[0]          # ← uendret X: bruker DVL-Z
    Yw = -x_dvl + t_post[1]
    Zw = z_world_from_depth + t_post[2]   # ← Z fra depth

    Rw = yaw   + rpy_offset[0]
    Pw = -roll + rpy_offset[1]
    Yw_r = -pitch + rpy_offset[2]

    xyz_out.append((Xw, Yw, Zw))
    rpy_out.append((Rw, Pw, Yw_r))
    time_out.append(t)

# ---------------- Skriv til fil ----------------
fmt = lambda v: f"{v:.4f}"
lines = []
for t, xyz, rpy in zip(time_out, xyz_out, rpy_out):
    line = (
        f'<keypoint time="{fmt(t)}" '
        f'xyz="{" ".join(fmt(v) for v in xyz)}" '
        f'rpy="{" ".join(fmt(v) for v in rpy)}"/>'
    )
    lines.append(line)

with open(output_file, "w") as f:
    f.write("\n".join(lines))

print(f"Wrote {len(lines)} keypoints to {output_file}")


Wrote 213 keypoints to keypoints.txt
